In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import torch
from model import *
from train import *

from test import test
import string
from utils import *
torch.manual_seed(1337)

In [40]:
def load_data(X,Y,block_size,stop_char_index):
    features = []
    target = []
    chars = []
    def s_e(text,i,block_size,return_last_char=False):
        start = i
        end = i + block_size
        start = min(len(text) - block_size,start)
        end = min(len(text),end)
        if return_last_char:
            return text[start:end],text[end] if end<len(text) else stop_char_index
        return text[start:end]
    for f,t in zip(X,Y):
        for i in range(len(f) - block_size):
            features.append(s_e(f,i,block_size))
            tt,c = s_e(t,i,block_size,return_last_char=True)
            target.append(tt)
            chars.append(c)
    features = torch.tensor(features)
    target = torch.tensor(target)
    chars = torch.tensor(chars)
    print(features.shape,target.shape,chars.shape)
    return features,target,chars

In [41]:
def prepare_data(
        src_path,
        dst_path,
        block_size,
        start_token="$",
        end_token = "^",
        add_start_end_tokens=True,
        device="cpu",
        num_lines=None,
        encoding="utf8",
    ):
    src = load_txt(src_path,num_lines,encoding=encoding)
    dst = load_txt(dst_path,num_lines,encoding=encoding)
    
    src_vocab = get_vocab(src) if not add_start_end_tokens else get_vocab(src,start=start_token,end=end_token)
    dst_vocab = get_vocab(dst) if not add_start_end_tokens else get_vocab(dst,start=start_token,end=end_token)

    src = [encode(txt,src_vocab) for txt in src]
    dst = [encode(txt,dst_vocab) for txt in dst]

    #src = list(filter(lambda l:len(l)==block_size,src))
    #dst = list(filter(lambda l:len(l)==block_size,dst))


    if add_start_end_tokens:
        src = include_tokens(
            src,
            start_token_index=src_vocab.index(start_token),
            amount=1
        )
        dst = include_tokens(
            dst,
            start_token_index=dst_vocab.index(start_token),
            amount=block_size-1
        )
    src,dst,chars = load_data(src,dst,block_size,dst_vocab.index(end_token))
    
    return src.to(device=device),src_vocab,dst.to(device=device),dst_vocab,chars

src_path = "./resources/news-commentary-v9.fr-en.en"
dst_path = "./resources/news-commentary-v9.fr-en.fr"

block_size = 5
src,src_vocab,dst,dst_vocab,chars = prepare_data(
    src_path,
    dst_path,
    block_size,
    add_start_end_tokens=True,
    num_lines=100,
)
encoder_vocab_size = len(src_vocab)
decoder_vocab_size = len(dst_vocab)
encoder_vocab_size,decoder_vocab_size,len(chars)

torch.Size([10297, 5]) torch.Size([10297, 5]) torch.Size([10297])


(52, 59, 10297)

In [15]:
decode(src[99],src_vocab)

' gold'

In [16]:
index,window =99,4
decode_all(src[index:index+window],src_vocab),decode_all(dst[index:index+window],dst_vocab)

([' gold', '$late', 'latel', 'ately'], ['eur d', '$$$$e', '$$$et', '$$et '])

In [57]:
N = 1
n_embeddings = 32
n_head = 4
device = "cpu"

model = AttentiveTranslator(
    3,
    encoder_vocab_size,
    decoder_vocab_size,
    n_embeddings,
    n_head,
    src.shape,
    dst.shape,
    device=device
).to(device=device)

In [45]:
def split(X,y,train_size=.8,shuffle=True):
    from sklearn.model_selection import train_test_split
    return train_test_split(X,y,chars,train_size=train_size,shuffle=shuffle)
src_train,src_valid,dst_train,dst_valid,chars_train,chars_val = split(src,dst)
len(src_train),len(src_valid),len(dst_train),len(dst_valid)

(8237, 2060, 8237, 2060)

In [19]:
src.shape,dst.shape

(torch.Size([10297, 5]), torch.Size([10297, 5]))

In [58]:
model = model.to(memory_format=torch.channels_last)
optim = torch.optim.Adam(
    model.parameters()
)
loss_fn = torch.nn.functional.cross_entropy
train_loader = arrays_to_loader(src_train,dst_train,chars_train,batch_size=32)
train(model,optim,loss_fn,1,train_loader,device,dst_vocab)

4.607472896575928
4.268762588500977
4.295784950256348
3.935128927230835
3.672642469406128
3.6724817752838135
3.3058464527130127
3.3122706413269043
3.1836607456207275
3.1797587871551514
3.4907479286193848
2.6631150245666504
3.49959397315979
3.0579090118408203
3.169254779815674
3.25533127784729
3.028381109237671
2.8590660095214844
3.407579183578491
2.9232866764068604
3.071563243865967
3.135579824447632
3.4837944507598877
2.642312526702881
3.3064870834350586
3.110717296600342
3.2464845180511475
3.1207122802734375
2.9501264095306396
3.1789543628692627
2.8416242599487305
2.7630410194396973
3.208578109741211
2.835777521133423
2.7957053184509277
2.93951153755188
3.159423351287842
3.176086187362671
2.7470169067382812
3.095226764678955
3.086824417114258
3.207108736038208
3.0458788871765137
2.5409460067749023
2.715014934539795
2.8203446865081787
2.6028475761413574
2.855919361114502
3.0887460708618164
2.548659563064575
2.6404359340667725
3.1062302589416504
3.0034141540527344
2.7324094772338867
3.

In [ ]:
from sklearn.metrics import accuracy_score
(dst_valid==test(model,src_valid,dst_valid)).float().mean()

tensor(0.9899)

In [59]:
from test import translate
translate(
    model,
    "hello there brother",
    block_size,
    src_vocab,
    dst_vocab,
    device
)

e
 
d
e
 
d
e
 
d
e
 
d
e
 


'e de de de de '

In [ ]:
from test import translate
translate(
    model,
    "ffffffffffffffffffffffffffffff",
    block_size,
    src_vocab,
    dst_vocab,
    device
)

tensor(125) tensor(90)
^^^^à
tensor(125) tensor(118)
^^^àà
tensor(125) tensor(146)
^^ààà
tensor(125) tensor(174)
^àààà
tensor(125) tensor(202)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà
tensor(125) tensor(230)
ààààà


'ààààààààààààààààààààààààà'